
# 🩺 医学因果推断 · 阶段 0（可运行 Notebook）
> 目标：完成环境准备、快速数据加载与基础探索（EDA），为后续因果推断实战打底。  
> 语言：Python  
> 适用方向：医学研究（RWD / EMR / 生存分析前置准备）

**建议学习路径**
1. 环境与依赖检查（含备选安装命令）  
2. 加载内置教学数据（DoWhy Lalonde）并做入门级探索  
3. 加载医学公共数据（UCI Heart Disease）并做入门级探索  
4. EDA 关注点（因果视角）与下一步路线



## 0. 环境与依赖检查（首次运行必看）

> 本单元不会强制联网安装依赖，而是先检查环境。  
> 如果缺包，请**在你本地的终端/Anaconda Prompt**中运行给出的 pip/conda 命令安装。


In [ ]:

import sys, platform, importlib

core_libs = [
    "pandas", "numpy", "matplotlib", "scikit_learn", "statsmodels",
    "dowhy", "econml", "causalml"
]

def check(lib):
    try:
        if lib == "scikit_learn":
            import sklearn as m
        else:
            m = importlib.import_module(lib)
        ver = getattr(m, "__version__", "unknown")
        print(f"[OK] {lib} - {ver}")
        return True
    except Exception as e:
        print(f"[MISSING] {lib} - {e}")
        return False

print("Python:", sys.version.split()[0], "| Platform:", platform.platform())
print("\n== Dependency Check ==")
missing = []
for lib in core_libs:
    ok = check(lib)
    if not ok:
        missing.append(lib)

if missing:
    print("\n== You seem to be missing the following packages ==")
    print(missing)
else:
    print("\nGreat! All recommended packages are available.")



### 若缺包，请在终端手动安装

> 建议在**独立虚拟环境**中安装，避免与其他项目冲突。

**Conda 创建环境（推荐）**
```bash
conda create -n causal python=3.10 -y
conda activate causal
```

**使用 pip 安装（在已激活的环境中）**
```bash
pip install -U pip
pip install pandas numpy matplotlib seaborn scikit-learn statsmodels dowhy econml causalml
```

> 注：`econml` 和 `causalml` 的编译依赖较多，Windows 环境推荐使用 Conda。若安装遇到困难，可优先保证 `pandas/numpy/matplotlib/scikit-learn/statsmodels/dowhy` 就能跑通本 Notebook 的主体内容。



## 1. 使用 DoWhy 内置数据（Lalonde）做“零门槛”上手

> 该数据集是因果推断教学经典案例（有**处理**与**对照**、有混杂变量），非常适合快速跑通流程。


In [ ]:

import pandas as pd
import numpy as np

try:
    import dowhy.datasets as dwd
    data = dwd.linear_dataset(
        beta=10,
        num_common_causes=5,
        num_instruments=1,
        num_samples=1000,
        treatment_is_binary=True,
        stddev_treatment_noise=1.0,
        stddev_outcome_noise=1.0,
        seed=2024
    )
    df_lalonde = data["df"]
    treatment_col = data["treatment_name"]
    outcome_col = data["outcome_name"]
    common_causes = data["common_causes_names"]
    instrument_names = data.get("instrument_names", [])
    display(df_lalonde.head())
except Exception as e:
    print("加载 DoWhy 内置数据失败：", e)
    df_lalonde = None
    treatment_col = outcome_col = None
    common_causes = instrument_names = None



### 1.1 快速了解数据结构


In [ ]:

if df_lalonde is not None:
    print("形状:", df_lalonde.shape)
    display(df_lalonde.describe(include='all'))
    print("\n列说明：")
    print("treatment:", treatment_col, "| outcome:", outcome_col)
    print("common_causes:", common_causes)
    print("instruments:", instrument_names)
else:
    print("未能加载数据。请先安装并重试。")



### 1.2 因果视角的基础 EDA（不求花哨，只求有效）
- **目标**：检查治疗组与对照组在关键基线变量上的差异（是否平衡）  
- **方法**：绘制直方图/箱线图；计算**标准化差异（Standardized Mean Difference, SMD）**


In [ ]:

import matplotlib.pyplot as plt

def standardized_mean_diff(x_t, x_c):
    # SMD for continuous variable
    mu_t, mu_c = np.nanmean(x_t), np.nanmean(x_c)
    sd_pooled = np.sqrt((np.nanvar(x_t, ddof=1)+np.nanvar(x_c, ddof=1))/2)
    return (mu_t - mu_c) / (sd_pooled + 1e-12)

if df_lalonde is not None:
    tmask = df_lalonde[treatment_col] == 1
    cmask = df_lalonde[treatment_col] == 0

    print("治疗组 n =", int(tmask.sum()), "| 对照组 n =", int(cmask.sum()))

    smd_rows = []
    for v in common_causes:
        x_t, x_c = df_lalonde.loc[tmask, v], df_lalonde.loc[cmask, v]
        smd = standardized_mean_diff(x_t, x_c)
        smd_rows.append((v, smd))
    smd_df = pd.DataFrame(smd_rows, columns=["variable", "SMD"]).sort_values("variable")
    display(smd_df)

    # 对其中一个变量画直方图
    var_to_plot = common_causes[0]
    plt.figure()
    plt.hist(df_lalonde.loc[tmask, var_to_plot], alpha=0.6, label="Treatment", bins=30)
    plt.hist(df_lalonde.loc[cmask, var_to_plot], alpha=0.6, label="Control", bins=30)
    plt.xlabel(var_to_plot); plt.ylabel("Count"); plt.legend(); plt.title(f"Distribution of {var_to_plot}")
    plt.show()
else:
    print("未能加载数据。请先安装并重试。")



## 2. 加载医学公共数据：UCI Heart Disease

> 该数据常用于心血管研究教学，适合用作**混杂明显的观测性数据**练习。  
> 下面的单元会尝试从公共仓库直接下载 CSV；若你本地无网络，请将数据下载到本地后，修改路径再运行。


In [ ]:

import pandas as pd

urls = [
    "https://raw.githubusercontent.com/selva86/datasets/master/Heart.csv",
]

df_heart = None
last_err = None
for url in urls:
    try:
        df_heart = pd.read_csv(url)
        print(f"已成功加载：{url}")
        break
    except Exception as e:
        last_err = e
        print("尝试加载失败：", url, "错误：", e)

if df_heart is None:
    print("\n未能在线加载数据，请手动下载 Heart.csv 并替换为你的本地路径重试。")
else:
    display(df_heart.head())
    print("形状:", df_heart.shape)
    display(df_heart.describe(include='all'))



### 2.1 因果视角下的入门 EDA（以“是否患病”为结局）
- 检查关键基线变量（如年龄、胆固醇、是否吸烟）在“患病/未患病”之间的差异  
- 为后续构建**因果图（DAG）**与**协变量选择**做准备


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

if df_heart is not None:
    possible_targets = [c for c in df_heart.columns if c.lower() in ("ahd", "target", "disease", "diagnosis")]
    target_col = possible_targets[0] if possible_targets else None
    print("识别到的可能结局列:", possible_targets, "| 使用：", target_col)

    if target_col is not None and df_heart[target_col].dtype == 'O':
        df_heart[target_col] = df_heart[target_col].astype('category').cat.codes

    candidate_baselines = [c for c in df_heart.columns if c != target_col]
    if target_col is not None:
        case_n = int((df_heart[target_col]==1).sum())
        ctrl_n = int((df_heart[target_col]==0).sum())
        print(f"病例 n={case_n} | 对照 n={ctrl_n}")

    def safe_numeric(col):
        try:
            return pd.to_numeric(df_heart[col], errors="coerce")
        except Exception:
            return None

    smd_rows = []
    if target_col is not None:
        case_mask = df_heart[target_col] == 1
        ctrl_mask = df_heart[target_col] == 0
        for col in candidate_baselines:
            x_num = safe_numeric(col)
            if x_num is None: 
                continue
            if x_num.notna().sum() < 5: 
                continue
            mu_t, mu_c = np.nanmean(x_num[case_mask]), np.nanmean(x_num[ctrl_mask])
            sd_pooled = np.sqrt((np.nanvar(x_num[case_mask], ddof=1)+np.nanvar(x_num[ctrl_mask], ddof=1))/2)
            smd = (mu_t - mu_c) / (sd_pooled + 1e-12)
            smd_rows.append((col, smd))
        import pandas as pd
        smd_tbl = pd.DataFrame(smd_rows, columns=["variable", "SMD"]).sort_values("variable")
        display(smd_tbl.head(20))

        numeric_cols = [c for c in candidate_baselines if pd.api.types.is_numeric_dtype(df_heart[c])]
        if numeric_cols:
            var_to_plot = numeric_cols[0]
            plt.figure()
            plt.hist(df_heart.loc[case_mask, var_to_plot].dropna(), bins=30, alpha=0.6, label="Cases")
            plt.hist(df_heart.loc[ctrl_mask, var_to_plot].dropna(), bins=30, alpha=0.6, label="Controls")
            plt.xlabel(var_to_plot); plt.ylabel("Count"); plt.legend(); plt.title(f"Distribution of {var_to_plot}")
            plt.show()
    else:
        print("未识别到结局列（AHD/target 等）。请手动指定 target_col 后重试。")



## 3. 因果视角下的 EDA 清单（医疗版）

- **治疗/暴露定义是否明确？**（例：是否使用某药、是否接受某项手术）  
- **结局定义是否明确？**（例：28 天死亡率、住院再入院、并发症发生）  
- **时间顺序明确吗？**（确保暴露先于结局）  
- **混杂因子列出了吗？**（年龄、性别、基线疾病、合并用药、入院严重度评分等）  
- **基线平衡如何？**（标准化差异 SMD 是否 > 0.1/0.2）  
- **缺失数据机制？**（MCAR/MAR/MNAR，计划如何填补或建模）  
- **极端值/异常值处理？**（winsorize、对数变换、分位数裁剪）  
- **抽样/选择偏倚迹象？**（纳入排除标准是否合理；外推性如何）



## 4. 下一步（衔接阶段 1）

1. 在 DoWhy 的 Lalonde 数据上构建简单的因果图（DAG），指定 `treatment → outcome`，并将常见混杂因子作为调整集。  
2. 使用倾向评分（逻辑回归）估计 ATE：PSM 或 IPTW，并观察**平衡性改善**。  
3. 做一次**敏感性分析**（如未测混杂的强度界值）。

> 如果你需要，我可以基于本 Notebook 继续生成**阶段 1 的可运行 Notebook**，直接带你完成第一次 ATE 估计与敏感性分析。
